<a href="https://colab.research.google.com/github/mjalalimanesh/IBM-SQL-DB2-Course/blob/main/DB0201EN-Week4-1-1-RealDataPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# Working with a real world data-set using SQL and Python

Estaimted time needed: **30** minutes

## Objectives

After complting this lab you will be able to:

-   Understand the dataset for Chicago Public School level performance
-   Store the dataset in an Db2 database on IBM Cloud instance
-   Retrieve metadata about tables and columns and query data from mixed case columns
-   Solve example problems to practice your SQL skills including using built-in database functions


## Chicago Public Schools - Progress Report Cards (2011-2012)

The city of Chicago released a dataset showing all school level performance data used to create School Report Cards for the 2011-2012 school year. The dataset is available from the Chicago Data Portal: [https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t](https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

This dataset includes a large number of metrics. Start by familiarizing yourself with the types of metrics in the database: [https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true](https://data.cityofchicago.org/api/assets/AAD41A13-BE8A-4E67-B1F5-86E711E09D5F?download=true&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)

**NOTE**: 

Do not download the dataset directly from City of Chicago portal. Instead download a static copy which is a more database friendly version from this <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_edX/data/Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012-v3.csv">link</a>.

Now review some of its contents.


### Store the dataset in a Table

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in the previous lab, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR. 

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next". 

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the CHICAGO PUBLIC SCHOOLS dataset and load the dataset into a new table called **SCHOOLS**.

<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/uc4xjh1uxcc78ks1i18v668simioz4es.jpg"></a>


In [1]:
!pip install ibm_db
!pip install ibm_db_sa

     |████████████████████████████████| 727kB 3.4MB/s 
  Created wheel for ibm-db: filename=ibm_db-3.0.2-cp36-cp36m-linux_x86_64.whl size=38977342 sha256=327a514fc76bc58b6a0568c3b679420917dd997564c7b29dd08457659848757f
  Stored in directory: /root/.cache/pip/wheels/84/64/42/10b217b545e8085b72160632c61bfa0909f661dad89326a956
Successfully built ibm-db
  Created wheel for ibm-db-sa: filename=ibm_db_sa-0.3.5-cp36-none-any.whl size=28000 sha256=ee6e9fb3df195ebec03b9faf6d25d8fb51def4ec36830fb275295f3e1e923ef1
  Stored in directory: /root/.cache/pip/wheels/7f/0b/94/ecb455e6d4042de08ee2f1b623a1e76d30e4e46f526d3763f3
Successfully built ibm-db-sa


In [1]:
import ibm_db
import ibm_db_sa

### Connect to the database

Let us now load the ipython-sql  extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
# Enter the connection string for your Db2 on Cloud database instance below
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name
%sql ibm_db_sa://bxd83998:9j6ngwcwqn0v8%40vs@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB

'Connected: bxd83998@BLUDB'

### Query the database system catalog to retrieve table metadata

##### You can verify that the table creation was successful by retrieving the list of all tables in your schema and checking whether the SCHOOLS table was created


In [8]:
# type in your query to retrieve list of all tables in the database for your db2 schema (username)
%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES where TABSCHEMA='BXD83998'


 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
BXD83998,CENSUS_DATA,2020-11-09 18:24:28.228846
BXD83998,CHICAGO_PUBLIC_SCHOOLS,2020-11-10 07:58:39.843671
BXD83998,CHICAGO_CRIME_DATA,2020-11-10 08:00:36.708883
BXD83998,INSTRUCTOR,2020-11-10 08:15:32.742185
BXD83998,INTERNATIONAL_STUDENT_TEST_SCORES,2020-11-10 08:54:09.841396
BXD83998,CHICAGO_SOCIOECONOMIC_DATA,2020-11-10 09:09:47.406478
BXD83998,EMPLOYEES,2020-10-16 17:38:33.256260
BXD83998,JOB_HISTORY,2020-10-16 17:38:33.378226
BXD83998,JOBS,2020-10-16 17:38:33.538166
BXD83998,DEPARTMENTS,2020-10-16 17:38:33.652184


In [5]:

# or, you can retrieve list of all tables where the schema name is not one of the system created ones:

%sql select TABSCHEMA, TABNAME, CREATE_TIME from SYSCAT.TABLES \
      where TABSCHEMA not in ('SYSIBM', 'SYSCAT', 'SYSSTAT', 'SYSIBMADM', 'SYSTOOLS', 'SYSPUBLIC')
      

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,create_time
BXD83998,CENSUS_DATA,2020-11-09 18:24:28.228846
BXD83998,CHICAGO_PUBLIC_SCHOOLS,2020-11-10 07:58:39.843671
BXD83998,CHICAGO_CRIME_DATA,2020-11-10 08:00:36.708883
BXD83998,INSTRUCTOR,2020-11-10 08:15:32.742185
BXD83998,INTERNATIONAL_STUDENT_TEST_SCORES,2020-11-10 08:54:09.841396
BXD83998,CHICAGO_SOCIOECONOMIC_DATA,2020-11-10 09:09:47.406478
BXD83998,EMPLOYEES,2020-10-16 17:38:33.256260
BXD83998,JOB_HISTORY,2020-10-16 17:38:33.378226
BXD83998,JOBS,2020-10-16 17:38:33.538166
BXD83998,DEPARTMENTS,2020-10-16 17:38:33.652184


In [10]:

# or, just query for a specifc table that you want to verify exists in the database
%sql select * from SYSCAT.TABLES where TABNAME = 'CHICAGO_PUBLIC_SCHOOLS'


 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


tabschema,tabname,owner,ownertype,TYPE,status,base_tabschema,base_tabname,rowtypeschema,rowtypename,create_time,alter_time,invalidate_time,stats_time,colcount,tableid,tbspaceid,card,npages,mpages,fpages,npartitions,nfiles,tablesize,overflow,tbspace,index_tbspace,long_tbspace,parents,children,selfrefs,keycolumns,keyindexid,keyunique,checkcount,datacapture,const_checked,pmap_id,partition_mode,log_attribute,pctfree,append_mode,REFRESH,refresh_time,LOCKSIZE,VOLATILE,row_format,property,statistics_profile,compression,rowcompmode,access_mode,clustered,active_blocks,droprule,maxfreespacesearch,avgcompressedrowsize,avgrowcompressionratio,avgrowsize,pctrowscompressed,logindexbuild,codepage,collationschema,collationname,collationschema_orderby,collationname_orderby,encoding_scheme,pctpagessaved,last_regen_time,secpolicyid,protectiongranularity,auditpolicyid,auditpolicyname,auditexceptionenabled,definer,oncommit,logged,onrollback,lastused,control,temporaltype,tableorg,extended_row_size,pctextendedrows,remarks
BXD83998,CHICAGO_PUBLIC_SCHOOLS,BXD83998,U,T,N,None,None,None,None,2020-11-10 07:58:39.843671,2020-11-10 07:58:39.843671,2020-11-10 07:58:39.843671,2020-11-10 08:07:51.921652,78,13,1143,566,15,0,16,-1,-1,-1,0,bxd83998space1,None,None,0,0,0,0,0,0,0,N,YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY,1,,0,-1,N,,None,R,,N,,None,N,,F,None,0,N,999,0,0.0,844,0.0,None,1208,SYSIBM,IDENTITY,SYSIBM,IDENTITY,,0,2020-11-10 07:58:39.843671,0,,None,None,N,BXD83998,,,,2020-11-10,,N,R,N,-1.0,None


### Query the database system catalog to retrieve column metadata

##### The SCHOOLS table contains a large number of columns. How many columns does this table have?


In [11]:
# type in your query to retrieve the number of columns in the SCHOOLS table
%sql select count(*) from SYSCAT.COLUMNS where TABNAME = 'CHICAGO_PUBLIC_SCHOOLS'

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


1
78


Now retrieve the the list of columns in SCHOOLS table and their column type (datatype) and length.


In [13]:
# type in your query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql select COLNAME, TYPENAME, LENGTH from SYSCAT.COLUMNS where TABNAME = 'CHICAGO_PUBLIC_SCHOOLS'

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


colname,typename,length
Phone_Number,VARCHAR,14
Link,VARCHAR,78
Network_Manager,VARCHAR,40
Collaborative_Name,VARCHAR,34
Adequate_Yearly_Progress_Made_,VARCHAR,3
Track_Schedule,VARCHAR,12
CPS_Performance_Policy_Status,VARCHAR,16
CPS_Performance_Policy_Level,VARCHAR,15
HEALTHY_SCHOOL_CERTIFIED,VARCHAR,3
Safety_Icon,VARCHAR,11


In [14]:
%sql select distinct(NAME), COLTYPE, LENGTH from SYSIBM.SYSCOLUMNS where TBNAME = 'CHICAGO_PUBLIC_SCHOOLS'

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


name,coltype,length
10th Grade PLAN (2009),VARCHAR,4
10th Grade PLAN (2010),VARCHAR,4
11th Grade Average ACT (2011),VARCHAR,4
9th Grade EXPLORE (2009),VARCHAR,4
9th Grade EXPLORE (2010),VARCHAR,4
AVERAGE_STUDENT_ATTENDANCE,VARCHAR,6
Adequate_Yearly_Progress_Made_,VARCHAR,3
Average_Teacher_Attendance,VARCHAR,6
COLLEGE_ENROLLMENT,SMALLINT,2
COMMUNITY_AREA_NAME,VARCHAR,22


### Questions

1.  Is the column name for the "SCHOOL ID" attribute in upper or mixed case?
2.  What is the name of "Community Area Name" column in your table? Does it have spaces?
3.  Are there any columns in whose names the spaces and paranthesis (round brackets) have been replaced by the underscore character "\_"?


## Problems

### Problem 1

##### How many Elementary Schools are in the dataset?


In [15]:
%sql SELECT COUNT(*) FROM CHICAGO_PUBLIC_SCHOOLS WHERE "Elementary, Middle, or High School" = 'ES'

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


1
462


### Problem 2

##### What is the highest Safety Score?


In [16]:
%sql SELECT MAX(SAFETY_SCORE) FROM CHICAGO_PUBLIC_SCHOOLS;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


1
99


### Problem 3

##### Which schools have highest Safety Score?


In [34]:
%sql SELECT "School_ID", NAME_OF_SCHOOL, SAFETY_SCORE FROM CHICAGO_PUBLIC_SCHOOLS \
   WHERE SAFETY_SCORE IS NOT NULL ORDER BY SAFETY_SCORE DESC LIMIT 25;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,safety_score
610038,Abraham Lincoln Elementary School,99
610230,Wildwood Elementary School,99
610249,Talman Elementary School,99
609880,Stephen Decatur Classical Elementary School,99
610155,Sauganash Elementary School,99
610105,Oriole Park Elementary School,99
610099,Norwood Park Elementary School,99
609744,Northside Learning Center High School,99
609749,Northside College Preparatory High School,99
610355,Mary E Courtenay Elementary Language Arts Center,99


### Problem 4

##### What are the top 10 schools with the highest "Average Student Attendance"?


In [33]:
%sql SELECT "School_ID", NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE FROM CHICAGO_PUBLIC_SCHOOLS  \
   ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC NULLS LAST LIMIT 10;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,average_student_attendance
609959,John Charles Haines Elementary School,98.40%
610217,James Ward Elementary School,97.80%
610132,Edgar Allan Poe Elementary Classical School,97.60%
610329,Orozco Fine Arts & Sciences Elementary School,97.60%
609842,Rachel Carson Elementary School,97.60%
610084,Annie Keller Elementary Gifted Magnet School,97.50%
610060,Andrew Jackson Elementary Language Academy,97.40%
610298,Lenart Elementary Regional Gifted Center,97.40%
610515,Disney II Magnet School,97.30%
610207,John H Vanderpoel Elementary Magnet School,97.20%


### Problem 5

##### Retrieve the list of 5 Schools with the lowest Average Student Attendance sorted in ascending order based on attendance


In [46]:
%sql SELECT "School_ID", NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE \
    FROM CHICAGO_PUBLIC_SCHOOLS \
    ORDER BY AVERAGE_STUDENT_ATTENDANCE NULLS LAST \
    FETCH FIRST 5 ROWS ONLY;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,average_student_attendance
609702,Richard T Crane Technical Preparatory High School,57.90%
609871,Barbara Vick Early Childhood & Family Center,60.90%
609736,Dyett High School,62.50%
609727,Wendell Phillips Academy High School,63.00%
610389,Orr Academy High School,66.30%


### Problem 6

##### Now remove the '%' sign from the above result set for Average Student Attendance column


In [51]:
%sql SELECT "School_ID", NAME_OF_SCHOOL, REPLACE(Average_Student_Attendance, '%', '') \
    FROM CHICAGO_PUBLIC_SCHOOLS \
    ORDER BY AVERAGE_STUDENT_ATTENDANCE NULLS LAST \
    FETCH FIRST 5 ROWS ONLY;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,3
609702,Richard T Crane Technical Preparatory High School,57.90
609871,Barbara Vick Early Childhood & Family Center,60.90
609736,Dyett High School,62.50
609727,Wendell Phillips Academy High School,63.00
610389,Orr Academy High School,66.30


### Problem 7

##### Which Schools have Average Student Attendance lower than 70%?


In [49]:
%sql SELECT "School_ID", NAME_OF_SCHOOL, AVERAGE_STUDENT_ATTENDANCE \
    FROM CHICAGO_PUBLIC_SCHOOLS \
    WHERE AVERAGE_STUDENT_ATTENDANCE < '70%' \
    ORDER BY AVERAGE_STUDENT_ATTENDANCE DESC NULLS LAST \
    FETCH FIRST 5 ROWS ONLY;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


School_ID,name_of_school,average_student_attendance
609759,Roberto Clemente Community Academy High School,69.60%
609674,Chicago Vocational Career Academy High School,68.80%
609722,Manley Career Academy High School,66.80%
610389,Orr Academy High School,66.30%
609727,Wendell Phillips Academy High School,63.00%


In [56]:
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from CHICAGO_PUBLIC_SCHOOLS \
     where CAST(REPLACE(Average_Student_Attendance, '%', '') AS DOUBLE ) < 70 \
     order by Average_Student_Attendance
     

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%
Manley Career Academy High School,66.80%
Chicago Vocational Career Academy High School,68.80%
Roberto Clemente Community Academy High School,69.60%


In [55]:
%sql SELECT Name_of_School, Average_Student_Attendance  \
     from CHICAGO_PUBLIC_SCHOOLS \
     where DECIMAL(REPLACE(Average_Student_Attendance, '%', '')) < 70 \
     order by Average_Student_Attendance


 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,average_student_attendance
Richard T Crane Technical Preparatory High School,57.90%
Barbara Vick Early Childhood & Family Center,60.90%
Dyett High School,62.50%
Wendell Phillips Academy High School,63.00%
Orr Academy High School,66.30%
Manley Career Academy High School,66.80%
Chicago Vocational Career Academy High School,68.80%
Roberto Clemente Community Academy High School,69.60%


### Problem 8

##### Get the total College Enrollment for each Community Area


In [64]:
%sql SELECT COMMUNITY_AREA_NAME, SUM(COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT_COUNT \
  FROM CHICAGO_PUBLIC_SCHOOLS \
  GROUP BY COMMUNITY_AREA_NAME \
  FETCH FIRST 5 ROWS ONLY;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,college_enrollment_count
ALBANY PARK,6864
ARCHER HEIGHTS,4823
ARMOUR SQUARE,1458
ASHBURN,6483
AUBURN GRESHAM,4175


### Problem 9

##### Get the 5 Community Areas with the least total College Enrollment  sorted in ascending order


In [75]:
%sql SELECT COMMUNITY_AREA_NAME, SUM(COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT_COUNT \
  FROM CHICAGO_PUBLIC_SCHOOLS \
  GROUP BY COMMUNITY_AREA_NAME \
  ORDER BY college_enrollment_count DESC \
  FETCH FIRST 5 ROWS ONLY;

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,college_enrollment_count
SOUTH LAWNDALE,14793
BELMONT CRAGIN,14386
AUSTIN,10933
GAGE PARK,9915
BRIGHTON PARK,9647


### Problem 10

##### Get the hardship index for the community area which has College Enrollment of 4368


In [85]:
%%sql 
SELECT COMMUNITY_AREA_NAME, SUM(COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT_COUNT
  FROM CHICAGO_PUBLIC_SCHOOLS 
  GROUP BY COMMUNITY_AREA_NAME
  HAVING SUM(COLLEGE_ENROLLMENT) = 4360 


 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,college_enrollment_count


In [69]:
%%sql 
select hardship_index 
   from chicago_socioeconomic_data CD, CHICAGO_PUBLIC_SCHOOLS CPS 
   where CD.ca = CPS.community_area_number 
      and college_enrollment = 4368


 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


hardship_index
6.0


### Problem 11

##### Get the hardship index for the community area which has the highest value for College Enrollment


In [106]:
%%sql
SELECT COMMUNITY_AREA_NAME, ca, hardship_index
  FROM chicago_socioeconomic_data
  WHERE ca IN ( SELECT COMMUNITY_AREA_NUMBER FROM
      (SELECT community_area_number, SUM(COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT_COUNT
        FROM CHICAGO_PUBLIC_SCHOOLS
        GROUP BY community_area_number
        ORDER BY college_enrollment_count DESC
        FETCH FIRST 5 ROWS ONLY)
  );
  

 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,ca,hardship_index
Belmont Cragin,19.0,70.0
Austin,25.0,73.0
South Lawndale,30.0,96.0
Brighton Park,58.0,84.0
Gage Park,63.0,93.0


Double-click **here** for the solution.

<!-- Solution:
NOTE: For this solution to work the CHICAGO_SOCIOECONOMIC_DATA table 
      as created in the last lab of Week 3 should already exist

%sql select ca, community_area_name, hardship_index from chicago_socioeconomic_data \
   where ca in \
   ( select community_area_number from schools order by college_enrollment desc limit 1 )

-->


In [122]:
%%sql 
SELECT CD.ca, MAX(hardship_index) AS hardship_index, SUM(SC.COLLEGE_ENROLLMENT) AS COLLEGE_ENROLLMENT_COUNT
  FROM CHICAGO_PUBLIC_SCHOOLS SC, chicago_socioeconomic_data CD
  WHERE CD.ca = SC.community_area_number
  GROUP BY CD.ca
  ORDER BY COLLEGE_ENROLLMENT_COUNT DESC
  LIMIT 5;


 * ibm_db_sa://bxd83998:***@dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net:50000/BLUDB
Done.


ca,hardship_index,college_enrollment_count
30.0,96.0,14793
19.0,70.0,14386
25.0,73.0,10933
63.0,93.0,9915
58.0,84.0,9647


## Summary

##### In this lab you learned how to work with a real word dataset using SQL and Python. You learned how to query columns with spaces or special characters in their names and with mixed case names. You also used built in database functions and practiced how to sort, limit, and order result sets, as well as used sub-queries and worked with multiple tables.


## Author

<a href="https://www.linkedin.com/in/ravahuja/" target="_blank">Rav Ahuja</a>

## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                 |
| ----------------- | ------- | ---------- | ---------------------------------- |
| 2020-08-28        | 2.0     | Lavanya    | Moved lab to course repo in GitLab |

<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
